In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

/home/teo/userdata/git_libraries/jgb-omr


In [2]:
from label_studio_sdk import Client
from pathlib import Path
from tqdm.auto import tqdm
import json
import random
import time


/home/teo/.local/share/virtualenvs/jgb-omr-AfDql38P/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
LABEL_STUDIO_URL = f'http://symphony2.sogang.ac.kr:21021/'
API_KEY = '922a5263681aa1818e57e84cdd0ea3dff9b5a1a0'
# Import the SDK and the client module
ls = Client(url=LABEL_STUDIO_URL, api_key=API_KEY)
ls.check_connection()
projects = ls.get_projects()


In [4]:
annotated_tasks = []
for project in projects:
  annotated_tasks += project.get_labeled_tasks()
len(annotated_tasks)

1179

In [5]:
def make_path_annotation_pair(task_results):
  pairs = []
  for task in task_results:
    annotations = task['annotations'][0]['result'][0]['value']['text'][0]
    png_name = task['storage_filename']
    png_name = Path(png_name).name
    pairs.append((png_name, annotations))
  return pairs

pairs = make_path_annotation_pair(annotated_tasks)
    

In [6]:
len(pairs)

1179

In [7]:
import pandas as pd

df = pd.DataFrame(pairs, columns=['Filename', 'Annotations'])
df.to_csv('pairs.csv', index=False)


In [8]:
df

,Filename,Annotations
0,관악영산회상-가락덜이_daegeum_37.png,청무\n-\n노라
1,관악영산회상-가락덜이_daegeum_58.png,청무\n-\n노
2,관악영산회상-가락덜이_daegeum_73.png,청임\n-\n같은음표 나니나
3,관악영산회상-가락덜이_haegeum_19.png,중 ㅅ
4,관악영산회상-가락덜이_haegeum_48.png,중 ㅅ
...,...,...
1174,여민락_piri_2149.png,"중 2\n-\n태, 황"
1175,여민락_piri_3022.png,남 서침표\n-\n임
1176,여민락_piri_3034.png,-\n-\n배남 ^
1177,여민락_piri_3094.png,- (\n-\n태)


In [17]:
pairs

[('/label-studio/files/jgb-pngs/관악영산회상-가락덜이_daegeum_37.png', '청무\n-\n노라'),
 ('/label-studio/files/jgb-pngs/관악영산회상-가락덜이_daegeum_58.png', '청무\n-\n노'),
 ('/label-studio/files/jgb-pngs/관악영산회상-가락덜이_daegeum_73.png',
  '청임\n-\n같은음표 나니나'),
 ('/label-studio/files/jgb-pngs/관악영산회상-가락덜이_haegeum_19.png', '중 ㅅ'),
 ('/label-studio/files/jgb-pngs/관악영산회상-가락덜이_haegeum_48.png', '중 ㅅ'),
 ('/label-studio/files/jgb-pngs/관악영산회상-가락덜이_piri_12.png', '무\n-\n황'),
 ('/label-studio/files/jgb-pngs/관악영산회상-가락덜이_piri_16.png', '- )\n-\n중'),
 ('/label-studio/files/jgb-pngs/관악영산회상-군악_daegeum_116.png',
  '청고 ^ :10 청태 :15\n'),
 ('/label-studio/files/jgb-pngs/관악영산회상-군악_daegeum_252.png', '청임 떠이어표\n청남 ^'),
 ('/label-studio/files/jgb-pngs/관악영산회상-군악_daegeum_262.png', '-\n청태 노니로'),
 ('/label-studio/files/jgb-pngs/관악영산회상-군악_daegeum_300.png', '청태 나니로'),
 ('/label-studio/files/jgb-pngs/관악영산회상-군악_daegeum_346.png', '-\n청태, 남'),
 ('/label-studio/files/jgb-pngs/관악영산회상-군악_daegeum_431.png', '임 ^\n니나'),
 ('/label-studio/files/jgb-pngs/관악영산

In [3]:
class TaskManager:
  def __init__(self, png_path_dir, task_json_dir, port=24594, host_url='163.239.15.26'):
    self.png_path_dir = png_path_dir

    self.task_json_dir = task_json_dir
    # self.task_stems, self.task_dict_list = self.get_task_json_file()
    self.port = port
    self.host_url = host_url

    
    # self.audio_not_in_task = list(set(self.audio_stems) - set(self.task_stems)) 
  
  def get_backup_labeled_tasks(self, json_path='label_results/annotated_tasks_backup_stems.json'):
    with open(json_path, 'r') as f:
      data = json.load(f)
    return data

  def assign_tasks(self, num_adding_task_per_person=100, margin=40):
    # 매번 task json을 불러오고 싶으면 아래 주석을 풀어주세요.
    # self.task_stems, self.task_dict_list = self.get_task_json_file()

    print(f"Total prepared tasks: {len(self.task_dict_list)}")
    ls = self.get_sdk_control()
    projects = ls.get_projects()

    annotator_candidates, num_unlabeled = self.get_annotators_to_add_tasks(projects, margin)
    if annotator_candidates == []:
      print("No annotator to add tasks")
      return
    assert len(annotator_candidates) == len(set(annotator_candidates)), "There is duplicated annotators"
    
    previously_assigned_stems = self.get_assigned_tasks_from_projects(projects)
    # add backuped labeled tasks
    with open('label_results/annotated_tasks_backup_stems.json') as f:
      backuped_stems = json.load(f)
    previously_assigned_stems += backuped_stems 
    previously_assigned_stems = list(set(previously_assigned_stems))

    # assert len(previously_assigned_stems) == len(set(previously_assigned_stems)), "There is duplicated files in the previously assigned files"
    print(f"Previously assigned tasks: {len(previously_assigned_stems)}")
    tasks_to_assign = self.get_tasks_that_are_not_assigned(previously_assigned_stems)
    print(f"Number of tasks to assign: {len(tasks_to_assign)}")
    random.shuffle(tasks_to_assign) # shuffle task order every time

    num_adding_tasks = len(annotator_candidates) * num_adding_task_per_person
    # num_adding_tasks = (len(annotator_candidates) + len(double_annotators)) * num_adding_task_per_person

    if len(tasks_to_assign) < num_adding_tasks:
      self.slack_bot.post_message(f"태스크가 부족하여 자동 할당을 정지합니다. 현재 할당 가능한 태스크 수: {len(tasks_to_assign)}")
    assert num_adding_tasks <= len(tasks_to_assign), "There are not enough tasks to assign"
    
    selected_tasks = tasks_to_assign[:num_adding_tasks]

    # adding_task_by_person = {}
    # current_slice_index = 0
    # for annotator in annotator_candidates:
    #   if annotator in double_annotators:
    #     num_tasks = num_adding_task_per_person * 2
    #   else:
    #     num_tasks = num_adding_task_per_person
    #   adding_task_by_person[annotator] = selected_tasks[current_slice_index:current_slice_index+num_tasks]
    #   current_slice_index += num_tasks

    adding_task_by_person = {annotator_candidates[i]: selected_tasks[i*num_adding_task_per_person:(i+1)*num_adding_task_per_person] for i in range(len(annotator_candidates))}
    total_task_stems = [Path(y['data']['audio']).stem for k, x in adding_task_by_person.items() for y in x]
    assert len(total_task_stems) == len(set(total_task_stems)), "There is duplicated files in the task jsons"
    assert set(previously_assigned_stems) - set(total_task_stems) == set(previously_assigned_stems), "There is duplicated tasks"

    # import tasks to annotators
    for project in projects:
      if project.title in adding_task_by_person.keys():
        project.import_tasks(adding_task_by_person[project.title])
        print(f"imported {len(adding_task_by_person[project.title])} tasks to {project.title}")
        time.sleep(5)
        is_safe = self.handle_duplicated_tasks(project)
        num_try = 0
        while not is_safe:
          time.sleep(2)
          is_safe = self.handle_duplicated_tasks(project)
          num_try += 1
          if num_try > 5:
            break
        len_unlabeled = len(project.get_unlabeled_tasks_ids())
        num_added_tasks = len_unlabeled - num_unlabeled[project.title]
        if num_added_tasks > num_adding_task_per_person:
          self.slack_bot.post_message(f"태스크가 중복돼서 추가되었습니다. {project.title}의 추가된 태스크 수: {num_added_tasks}")


    self.slack_bot.post_message(f"태스크 할당이 완료되었습니다. 남은 태스크 수: {len(tasks_to_assign) - num_adding_tasks}, 추가한 레이블러: {'/ '.join(annotator_candidates)}")
    
  def get_annotators_to_add_tasks(self, projects, margin=40, barline=100):
    num_unlabeled = {x.title: len(x.get_unlabeled_tasks_ids()) for x in projects}
    num_labeled = {x.title: len(x.get_labeled_tasks_ids()) for x in projects}

    annotator_candidates = []
    # double_annotators = []
    for k, v in num_unlabeled.items():
      if k in self.dropped_annotators:
        continue
      if num_labeled[k] < 2 and v > 10:
        continue
      elif v < margin:
        annotator_candidates.append(k)
        # if num_labeled[k] > 1000:
        #   double_annotators.append(k)
    return annotator_candidates, num_unlabeled


    # annotator_candidates = [key for key, value in num_unlabeled.items() 
    #                         if value < margin 
    #                           # or (num_labeled[key] >= barline and value < margin*2)
    #                         ]
    # return annotator_candidates


  def get_task_json_file(self):# get task json file:
    task_jsons = list(Path(self.task_json_dir).glob('*.json'))
    task_stems = []
    task_dict_list = []
    
    for fn in task_jsons:
      with open(fn, 'r') as f:
        data = json.load(f)
      # get audio file name:
      task_stems += self.get_valid_stems(data)
      task_dict_list += [x for x in data if self._check_is_valid(x)]
    
    assert len(task_stems) == len(set(task_stems)), "There is duplicated files in the task jsons"
    return task_stems, task_dict_list
  
  def get_valid_stems(self, alist):
    return [Path(x['data']['audio']).stem for x in alist if self._check_is_valid(x)]
  
  def _check_is_valid(self, task_dict):
    if ' ' in task_dict['data']['audio']:
      return False
    elif len(task_dict['data']['text'].split(':')[1]) < 2:
      return False
    else:
      return True
    
  def get_sdk_control(self):
    LABEL_STUDIO_URL = f'http://{self.host_url}:{self.port}/'
    API_KEY = '922a5263681aa1818e57e84cdd0ea3dff9b5a1a0'
    # Import the SDK and the client module
    ls = Client(url=LABEL_STUDIO_URL, api_key=API_KEY)
    ls.check_connection()
    return ls
  
  def get_tasks_that_are_not_assigned(self, previously_assigned_stems, save=True, out_name='remaining_tasks.json'):  
    tasks_to_assign = []
    for task in self.task_dict_list:
      if Path(task['data']['audio']).stem not in previously_assigned_stems:
        tasks_to_assign.append(task)
    if save:
      with open(out_name, 'w') as f:
        json.dump(tasks_to_assign, f, ensure_ascii=False, indent=2)
    return tasks_to_assign

  def get_assigned_tasks_from_projects(self, projects):
    previously_assigned_stems = []
    print("Loading assigned tasks from projects")
    for project in tqdm(projects):
      assigned_tasks = project.get_tasks()
      previously_assigned_stems += [Path(x['data']['audio']).stem for x in assigned_tasks]
    return previously_assigned_stems
  
  def delete_unassigned_projects(self):
    ls = self.get_sdk_control()
    projects = ls.get_projects()

    for project in projects:
      if '---' in project.title and len(project.get_labeled_tasks_ids()) == 0:
        ls.delete_project(project.id)
        print(f"Deleted {project.title}")
        
        
  def get_labeled_tasks(self, save=True, out_name='annotated_tasks.json'):
    ls = self.get_sdk_control()
    projects = ls.get_projects()
    annotated_tasks = []
    for project in projects:
      annotated_tasks += project.get_labeled_tasks()
    if save:
      with open(out_name, 'w') as f:
        json.dump(annotated_tasks, f, ensure_ascii=False, indent=2)
    return annotated_tasks

  def handle_duplicated_tasks(self, project):
    unlabeled_tasks = project.get_unlabeled_tasks()
    task_files = [x['data']['audio'] for x in unlabeled_tasks]
    task_files = [x for x in task_files if x != 'error']
    if len(set(task_files)) != len(task_files): # there is duplicated files
      print(f"Duplicated task added to {project.title}")
      duplicated_files = [x for x in set(task_files) if task_files.count(x) > 1]
      task_ids = [x['id']for x in unlabeled_tasks]
      for file in duplicated_files:
        idx = task_files.index(file)
        task_id = task_ids[idx]
        new_data = {'audio': 'error', 'text': '중복된 파일 에러: 리포트 해주세요'}
        project.update_task(task_id, data=new_data)
      time.sleep(2)
      self.slack_bot.post_message(f"{project.title}에 태스크가 중복되어 추가되었습니다. 확인해서 삭제해주세요.")
      return False
    else:
      return True



In [5]:
manager = TaskManager(png_path_dir='pngs', task_json_dir='task_jsons', port=24594)
projects = manager.get_sdk_control().get_projects()

In [6]:
project = projects[0]
# assert project.title == '정간보 테스트'


In [7]:
annotated_tasks = []
for project in projects:
  annotated_tasks += project.get_labeled_tasks()


In [10]:
len(annotated_tasks)

299

In [20]:
pairs[10]

('/label-studio/files/jgb-pngs/관악영산회상-군악_haegeum_499.png', ['남 덧길이\n임 반길이'])

In [14]:
annotated_tasks[10]

{'id': 18,
 'predictions': [],
 'annotations': [{'id': 212,
   'created_username': ' bobo7806@naver.com, 4',
   'created_ago': '21\xa0minutes',
   'completed_by': 4,
   'result': [{'value': {'text': ['남 덧길이\n임 반길이']},
     'id': 'HjDAxkurZG',
     'from_name': 'caption',
     'to_name': 'image',
     'type': 'textarea',
     'origin': 'manual'}],
   'was_cancelled': False,
   'ground_truth': False,
   'created_at': '2023-12-04T06:30:38.095916Z',
   'updated_at': '2023-12-04T06:30:38.096002Z',
   'draft_created_at': None,
   'lead_time': 48.967,
   'import_id': None,
   'last_action': None,
   'task': 18,
   'project': 1,
   'updated_by': 4,
   'parent_prediction': None,
   'parent_annotation': None,
   'last_created_by': None}],
 'drafts': [],
 'annotators': [4],
 'inner_id': 18,
 'cancelled_annotations': 0,
 'total_annotations': 1,
 'total_predictions': 0,
 'completed_at': '2023-12-04T06:30:38.095916Z',
 'annotations_results': '[{value: {text: [\\ub0a8 \\ub367\\uae38\\uc774\\n\\uc784 

In [ ]:
# pip install easyocr
